## Introduction and Scope

This notebook is a Jupyter Notebook for interactive development of the Dash visualization app that will accompany the Subject to Change distribution.

The app is/will be deployed in a different repository, so that it can have a standalone webpage. However, it is much faster to tweak the code in a Notebook than on the terminal.

In [35]:
import pandas as pd
import numpy as nd
import matplotlib.pyplot as plt
from matplotlib import cm
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

## Load the data:

In [44]:
group_years = pd.read_csv("./data/group_years_regions.csv")

In [50]:

print(group_years.shape)
print(group_years.columns)


(2180, 22)
Index(['Unnamed: 0', 'ucdp_name', 'ucdp_dset_id', 'year', 'modeledarticles',
       'countT1', 'countT2', 'propT1', 'propT2', 'propdiff', 'propdif.L1',
       'propdif.L2', 'delta1', 'delta1.5', 'delta2', 'delta1_L2', 'gap25',
       'gap50', 'counter', 'frexWords', 'side_b_dset_id', 'region'],
      dtype='object')
<bound method NDFrame._add_numeric_operations.<locals>.min of 0       1990
1       1991
2       1992
3       1993
4       1994
        ... 
2175    2010
2176    2020
2177    2019
2178    2019
2179    2020
Name: year, Length: 2180, dtype: int64>
<bound method NDFrame._add_numeric_operations.<locals>.max of 0       1990
1       1991
2       1992
3       1993
4       1994
        ... 
2175    2010
2176    2020
2177    2019
2178    2019
2179    2020
Name: year, Length: 2180, dtype: int64>


## Create helper objects:

In [47]:
tst = group_years.ucdp_name.unique().tolist()

print(tst[:10])
print(tst[0])

['KDPI', 'CPP', 'KNU', 'RSO', 'NMSP', 'PWG', 'MCC', 'CPI-Maoist', 'KIO', 'PFLP']
KDPI


## Test a way to automatically extract FREX words

(This works, but will need a few case statements in case there are no words attached)

In [28]:
grup1 = tst[0]

df = group_years

print

cond1 = df["propT1"] > 0.5
cond2 = df[df["ucdp_name"] == tst[1]]

print(df.loc[(df['ucdp_name'] == (tst[0])) & (df.propT1 > 0.5)].frexWords[0])
#print(df.loc[(df['ucdp_name'] == (tst[3])) & (df.propT2 > 0.5)].frexWords[0])

#T1_label = df[(df.propT1 > 0.5) & (df.loc["ucdp_name"] == "CPP")].frexWords[0]


print(tst[0])
print(tst[1])
print(tst[2])


r,opposit,hengaw,link,men,organ,re,suspicion,human,two
KDPI
CPP
KNU


## Dev Interactive Dash Plots

1) Basic plot:

In [29]:

import dash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Group Trajectory Visualizer (Dev)'),
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id="checklist",
        #options=['KDPI', 'AQAP', 'KNU', 'CPP'],
        options = tst[:10],
        value=[tst[0]],
        inline=True
    ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))

def update_line_chart(value):
  
    df = group_years.sort_values(by="year") # replace with your own data source
    mask = df.ucdp_name.isin(value)
    
    fig = px.line(df[mask], 
                     x="year", 
                     y="propdiff", 
                    color="ucdp_name", 
                 labels={"year_total": "Number of Changes",
                        "year": "Year",
                        'ucdp_name' : "Selected Group/s"})
    fig.add_hline(y=0.0, line_dash="dash", line_color="green")
    fig.update_layout(yaxis_range=[-1.05,1.05])
    fig.update_layout(plot_bgcolor='white')
    fig.update_xaxes(
    mirror=True,
    ticks='outside',
    dtick=1,
    showline=True,
    linecolor='black',
   # gridcolor='lightgrey'
)
    fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    tick0 =1,
    dtick=1
    #gridcolor='lightgrey'
)
    
    return fig


app.run_server(debug=True)

## Developing For Regions

(The goal is to eventually have an interacative plot where users can select the region, which will customize the drop-down list of potential groups to compare).

Alternatively, a version with multiple regions presented

In [18]:
## Do the regions by hand:

tst_asia = group_years[group_years["region"] == "Asia"].ucdp_name.unique().tolist()

app = Dash(__name__)


app.layout = html.Div(children=[
    html.H4('Groups in Asia (Dev)'),
    dcc.Graph(id="graph"),
    
    html.Div([
        "Select Group: ",
        dcc.Dropdown(
        id="dropdown",
        options = tst_asia,
        value=[tst_asia[0]],
        #inline=True
    ),
    ]),
    
    html.Div([
        "Select Region ",
        dcc.RadioItems(id='region-radio',
        options=[
        {'label': 'Asia', 'value': 'Asia'},
        {'label': 'Africa', 'value': 'Africa'},
        {'label': 'Americas', 'value': 'All'},
        {'label': 'Europe', 'value': 'Europe'},
        {'label': 'Middle East', 'value': 'Middle East'},
        {'label': 'Multiple', 'value': 'Multiple'}
            ],
        value='All',
        style={'width': '20%', 'display': 'inline-block'}),
         ]),

]) #closes app.layout call

@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))

def update_line_chart(value):
  
    df = group_years[group_years["region"] == "Asia"].sort_values(by="year") ## sorted by year so line works
    mask = df.ucdp_name.isin([value])
    
    fig = px.line(df[mask], 
                     x="year", 
                     y="propdiff", 
                  color="ucdp_name", 
                 labels={
                     "propdiff": "Frame Summary",
                     "udcp_name": "Group Name",
                     "year": "Year", 
                     'ucdp_name' : "Selected Group"
                 })
    fig.add_hline(y=0.0, line_dash="dash", line_color="green")
    fig.update_layout(yaxis_range=[-1,1])
    fig.update_layout(plot_bgcolor='white')
    return fig


app.run_server(debug=True)

In [59]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd

# Sample data
df=group_years

# Initialize the Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown-category1',
        options=[{'label': i, 'value': i} for i in df['region'].unique()],
        value=df['region'].unique()[0]
    ),
    html.Br(),
    dcc.Dropdown(id='dropdown-category2'),
    html.Br(),
    dcc.Graph(id='line-plot')
])

# Define callback to update dropdown options based on category1 selection
@app.callback(
    Output('dropdown-category2', 'options'),
    [Input('dropdown-category1', 'value')]
)
def update_dropdown_category2(selected_category1):
    filtered_df = df[df['region'] == selected_category1]
    options = [{'label': i, 'value': i} for i in filtered_df['ucdp_name'].unique()]
    return options

# Define callback to update line plot based on category2 selection
@app.callback(
    Output('line-plot', 'figure'),
    [Input('dropdown-category1', 'value'),
     Input('dropdown-category2', 'value')]
)
def update_line_plot(selected_category1, selected_category2):
    filtered_df = df[(df['region'] == selected_category1) & (df['ucdp_name'] == selected_category2)]
    filtered_df = filtered_df.sort_values(by="year") ## filter for line sanity, new line for readiability
    
    fig = px.line(filtered_df, 
                     x="year", 
                     y="propdiff", 
                  color="ucdp_name", 
                 labels={
                     "propdiff": "Frame Summary",
                     "udcp_name": "Group Name",
                     "year": "Year", 
                     'ucdp_name' : "Selected Group"
                 })
    fig.update_xaxes(range=[1991, 2020])
    fig.add_hline(y=0.0, line_dash="dash", line_color="green")
    fig.update_layout(yaxis_range=[-1,1],  
                      xaxis={'dtick': 1},
                      
                     )
    fig.update_layout(plot_bgcolor='white')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

